###   lengths summary:

In [2]:
# record

3     492
4     451
5     380
6     318
7     292
8     278
9     200
10    169
11    155
12    116
13     89
14     99
15     89
16     62
17     42
18     43
19     18
20     14
21      7
22      1
25      2
Name: length, dtype: int64

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math
import csv
import matplotlib.pyplot as plt
import datetime
#from datetime import datetime, date
import time
import os

In [2]:
fpath_id = './save_features/other/0_uid.csv'
df_id = pd.read_csv(fpath_id, header=None)
df_id.columns = ['uid']

In [3]:
other_list = os.listdir('./save_features/other')
other_list.sort()
other_list[:3]

['0000_forcast_diab.csv', '0_uid.csv', '10_earlobe.csv']

In [72]:
df_yr = pd.read_csv('./save_features/other/4_yr.csv', header=None)
df_yr.columns = ['yr']
df_mon = pd.read_csv('./save_features/other/5_mon.csv', header=None)
df_mon.columns = ['mon']
df_day = pd.read_csv('./save_features/other/6_day.csv', header=None)
df_day.columns = ['day']

In [73]:
df_date = pd.concat([df_yr, df_mon, df_day], axis = 1)
print(df_date.shape)
df_date.head(2)

(25142, 3)


,yr,mon,day
0,2005,3,12
1,2006,10,21


In [74]:
date_series = pd.to_datetime(dict(year=df_date.yr, month=df_date.mon, day=df_date.day))

In [75]:
date_series = pd.to_datetime(dict(year=df_date.yr, month=df_date.mon, day=df_date.day))
#365*24*60*60 = 31536000
df_dt_365 = pd.DataFrame(date_series.diff().dt.total_seconds().fillna(0)/31536000.)
df_dt_365.columns = ['del_yr']
df_dt_365.head(1)

,del_yr
0,0.0


In [8]:
df_dt_365[  (df_id.shift(1)!=df_id)['uid']    ] = 0
df_dt_365.head(1)

,del_yr
0,0.0


In [9]:
df_forcast_365 = df_dt_365.shift(-1)
df_forcast_365['del_yr'][len(df_forcast_365)-1] = 0
df_forcast_365.columns = ['forcast_yr']
print( df_forcast_365['forcast_yr'][len(df_forcast_365)-1]   )

0.0


In [10]:
#df_dt_365.to_csv('./save_features/numerical/00_del_yr.csv', index=False,  header=None)
df_forcast_365.to_csv('./save_features/numerical/000_forcast_yr.csv', 
                      index=False, header=None)

In [11]:
fpath_ans = './save_features/other/3_diabetes.csv'
df_ans = pd.read_csv(fpath_ans, header=None)
df_ans.columns = ['diabetes'] 

In [12]:
df_forcast_diabetes = df_ans.shift(-1)
df_forcast_diabetes[  (df_id.shift(-1)!= df_id)['uid']    ] = df_ans[  (df_id.shift(-1)!= df_id)['uid']    ]
df_forcast_diabetes['diabetes'][len(df_forcast_diabetes) - 1] = df_ans['diabetes'][len(df_forcast_diabetes) - 1]
df_forcast_diabetes.columns = ['forcast_diab']

In [13]:
df_forcast_diabetes.to_csv('./save_features/other/0000_forcast_diab.csv',
                           index=False, header=None)

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math
import csv
import matplotlib.pyplot as plt
#import datetime
#from datetime import datetime, date
import time
import os
t1 = time.time()

fpath_id = './save_features/other/0_uid.csv'
df_id = pd.read_csv(fpath_id, header=None)
df_id.columns = ['uid']

#fpath_ans = './save_features/other/3_diabetes.csv'
fpath_ans = './save_features/other/0000_forcast_diab.csv'
df_ans = pd.read_csv(fpath_ans, header=None)
# df_ans.columns = ['diabetes'] 
df_ans.columns = ['forcast_diab'] 

#df_tar = pd.get_dummies(df_ans['diabetes'])
df_tar = pd.get_dummies(df_ans['forcast_diab'])
df_tar.columns = ['no','yes']

fpath_num = './save_features/numerical/'
fpath_cate = './save_features/categorical/'

num_list = os.listdir(fpath_num)[:50]
cate_list = os.listdir(fpath_cate)[:30]

df_n = pd.read_csv(fpath_num+num_list[0], header=None)
df_n.columns = [num_list[0][4:-4]] ### needs regular expression
for name in num_list[1:]:
    df_n_new = pd.read_csv(fpath_num + name, header=None)
    df_n_new.columns = [name[4:-4]]
    df_n = pd.concat([df_n,  df_n_new], axis=1)
    
df_c = pd.read_csv(fpath_cate+cate_list[0])#, header=None)
#df_c.columns = [cate_list[0][4:-4]] ### needs regular expression
cate_ft_len = [len(df_c.columns)]
for name in cate_list[1:]:
    df_c_new = pd.read_csv(fpath_cate + name)#, header=None)
    cate_ft_len.append( len(df_c_new.columns)  )
    #df_c_new.columns = [name[4:-4]]
    df_c = pd.concat([df_c,  df_c_new], axis=1)
    
num_ft_len = [len(df_n.columns)]
ft_len_list = num_ft_len + cate_ft_len

df_total = pd.concat([df_n,  df_c], axis=1)

#######  df_len  should be saved and loaded, not calculated everytime!!
length = []
for id_ in set(df_id.uid):
    length.append(len(df_id[df_id['uid']==id_]))
df_len_1 = pd.DataFrame(list(set(df_id.uid)))
df_len_1.columns = ['uid'] 
df_len_2 = pd.DataFrame(length)
df_len_2.columns = ['length']
df_len = pd.concat([df_len_1,  df_len_2], axis=1)

batch_list = []
for ii in range(26):
    batch_list.append([])
for len_ in range(3, 23):
    batch_list[len_].append( df_len[df_len.length==len_]    )
batch_list[25].append( df_len[df_len.length==25]    )

batches = []
targets = []
for ii in range(26):
    batches.append([])
    targets.append([])    
for idx_len in range(3,19):
    for id_ in batch_list[idx_len][0]['uid']:
        batches[idx_len].append(np.array(  df_total[df_id['uid'] == id_] ) )
        targets[idx_len].append(np.array(  df_ans[df_id['uid'] == id_] ) )   


record = df_len['length'].value_counts().sort_index()
num_samples = 0
max_len = 18
for idx in range(3, max_len+1):
    num_samples += record[idx]
print('%d samples.'%num_samples)

time.time() - t1

3275 samples.


16.442413806915283

In [2]:
LEARNING_RATE = 1.0
MAX_GRAD_NORM = 1
KEEP_PROB = 0.8

max_len = 18
#NUM_FT = np.array(batches[3]).shape[2]  ## NUM_FT = 50
NUM_FT = sum(ft_len_list)
###  ft_len_list  = num_ft_len + cate_ft_len

HIDDEN_SIZE = 50
TAR_SIZE = 1
CLASS_SIZE = 2
NUM_LAYERS = 2

NUM_EPOCH = 500
TRAIN_BATCH_SIZE = 32
TRAIN_NUM_STEP = max_len
epoch_size = int(num_samples/TRAIN_BATCH_SIZE )

EVAL_BATCH_SIZE = 1
EVAL_NUM_STEP = 1

In [3]:
train_X = np.zeros([num_samples, max_len, NUM_FT])
train_Y = np.zeros([num_samples, max_len, 1])
train_lengths = np.zeros([num_samples])

idx_i = 0
for idx in range(3, max_len+1):
    idx_f = idx_i + record[idx]
    train_X[idx_i: idx_f, :idx, :] = np.array(  batches[idx]  )
    train_Y[idx_i: idx_f, :idx, :] = np.array(  targets[idx]  )
    train_lengths[idx_i: idx_f] = idx
    idx_i = idx_f

print(train_X.shape, train_Y.shape, train_lengths.shape)

### Shuffle:
indices = np.array(range(num_samples) )
np.random.shuffle(indices)
train_X = train_X[indices]
train_Y = train_Y[indices]
train_lengths = train_lengths[indices]

train_data = [train_X, train_Y, train_lengths]

((3275, 18, 149), (3275, 18, 1), (3275,))


In [4]:
def one_batch(raw_data, batch_size, num_steps, NUM_FT, epoch_size):  
    data_x = tf.convert_to_tensor(raw_data[0], name="raw_data",
                                      dtype=tf.float32)
    data_y = tf.convert_to_tensor(raw_data[1], name="raw_data",
                                      dtype=tf.int32)  
    data_l = tf.convert_to_tensor(raw_data[2], name="raw_data",
                                      dtype=tf.int32)  

    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = tf.strided_slice(data_x, [i*batch_size, 0, 0],
                         [(i + 1)*batch_size, num_steps, NUM_FT], [1,1,1])
    y = tf.strided_slice(data_y, [i*batch_size, 0, 0], 
                         [(i + 1)*batch_size, num_steps, 1], [1,1,1])
    l = tf.strided_slice(data_l, [i*batch_size], 
                         [(i + 1)*batch_size], [1])
    x.set_shape([batch_size, num_steps, NUM_FT])
    y.set_shape([batch_size, num_steps, 1])
    l.set_shape([batch_size])
    
    return x, y, l

In [5]:
def run_epoch(session, model, batch_data, train_op, output_log, epoch_size):
    total_costs = 0.0
    iters = 0
    state = session.run(model.initial_state)

    for step in range(epoch_size):
        x, y, length = session.run(batch_data)
        cost, state, _ = session.run([model.cost,
                                      model.final_state,
                                      train_op],
                                   feed_dict ={model.input_data: x,
                                    model.targets: y,
                                    model.seq_len: length })
                                   # model.initial_state: state})
        total_costs += cost
        iters += model.num_steps

        #if output_log and step % 40  == 0:
            #print("After %d steps, loss is %.3f" % (step, total_costs / iters))
            #print("After %d steps, perplexity is %.3f" % (step, np.exp(total_costs / iters)))
    return np.exp(total_costs / iters)

In [6]:
class Diab_Model(object):
    def __init__(self, is_training, batch_size, num_steps,
                                   ft_size, tar_size, HIDDEN_SIZE):
        '''
        ** ft_size is a list,like this:
         [100, 3, 2, 4, ...]. ft_len_list[0] is number of numerical features,
         ft_len_list[1:] correspond to number of class for each one-hot encoding feature.  
        '''
        
        self.batch_size = batch_size
        self.num_steps = num_steps
        NUM_FT = sum(ft_size)
        self.n_ft_size = ft_size[0]
        self.tar_size = tar_size 
        self.hidden_size = HIDDEN_SIZE

        ## Define LSTM layers, use "Dropout" when training:
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell( self.hidden_size  )
        #lstm_cell = tf.contrib.rnn.BasicLSTMCell( self.hidden_size )        
        if is_training:
            lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, 
                                        output_keep_prob=KEEP_PROB)
            #lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, 
            #                            output_keep_prob=KEEP_PROB)
        cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell]*NUM_LAYERS)
        #cell = tf.contrib.rnn.MultiRNNCell([lstm_cell]*NUM_LAYERS)

        ## Set weights and bias:
        self.n_ft_weight = tf.get_variable("n_ft_weight", [self.n_ft_size, self.hidden_size])
        self.n_ft_bias = tf.get_variable("n_ft_bias", [self.hidden_size])
        
        self.weight = tf.get_variable("weight", [self.hidden_size, CLASS_SIZE])
        self.bias = tf.get_variable("bias", [CLASS_SIZE])
        
        '''
        weights: coding here
        '''
        
        
        ## Set inputs:
        #inputs = self.input_data
        self.input_data = tf.placeholder(tf.float32, 
                                [batch_size, num_steps, NUM_FT ])
        self.targets = tf.placeholder(tf.int32, 
                                [batch_size, num_steps, tar_size ])
        self.seq_len = tf.placeholder(tf.int32, [batch_size])
        self.initial_state = cell.zero_state(
                             batch_size, tf.float32)
#        self.initial_state = cell.zero_state(
#                          [batch_size], tf.float32)
        states = self.get_state_variables(batch_size, cell)
        
        '''
        inputs = tf.reshape( self.input_data, [-1, ft_size])
        inputs = tf.matmul(inputs, self.n_ft_weight) + self.n_ft_bias
        inputs = tf.reshape( inputs, [batch_size, num_steps, self.hidden_size])
        '''
        n_inputs = tf.strided_slice(self.input_data, [0, 0, 0],
                     [batch_size, num_steps, self.n_ft_size], [1,1,1])
        self.n_inputs  = tf.reshape( n_inputs , [-1, self.n_ft_size])
        self.n_latent  = tf.matmul(self.n_inputs , self.n_ft_weight) + self.n_ft_bias
        #n_latent  = tf.reshape(n_latent, [batch_size, num_steps, self.hidden_size])
        
        self.c_inputs = tf.strided_slice(self.input_data, [0, 0, self.n_ft_size],
                     [batch_size, num_steps, NUM_FT], [1,1,1])
        
        c_weights = []
        for idx in range(1, len(ft_size) ) :
            embed_sz = ft_size[idx]/2
            name = 'c_weight_' + str(idx)
            c_weights.append( self.get_weights(
                             ft_size[idx], embed_sz, name) )  

        c_rk2_inputs = tf.reshape(self.c_inputs , [-1, NUM_FT-self.n_ft_size])
        c_one_input = tf.strided_slice(c_rk2_inputs, [0, 0],
                            [batch_size*num_steps, ft_size[1] ], [1,1])
        c_latent = tf.matmul(c_one_input, c_weights[0][0] ) + c_weights[0][1] 
        idx_i = ft_size[1]
        
        for idx in range(2, len(ft_size) ):
            idx_f = idx_i + ft_size[idx]
            c_one_input = tf.strided_slice(c_rk2_inputs, [0, idx_i],
                            [batch_size*num_steps, idx_f ], [1,1])
            c_one_latent = tf.matmul(c_one_input, c_weights[idx-1][0] 
                                ) + c_weights[idx-1][1]  
            idx_i = idx_f
            # c_latent = tf.concat([c_latent, c_one_latent], axis=1) ##for py35
            c_latent = tf.concat(1, [c_latent, c_one_latent]) ##for py27
            
        self.c_latent = c_latent  

        #c_latent  = tf.reshape(c_latent, [batch_size, num_steps,
        #                                  tf.shape(c_latent)[1]])

        #total_inputs = tf.concat([n_latent, c_latent], axis=1) ##for py35
        total_inputs = tf.concat(1, [self.n_latent, c_latent]) ##for py27
        total_inputs = tf.reshape(total_inputs,
                    [batch_size, num_steps, tf.shape(total_inputs)[1]] )
  
        if is_training:
            total_inputs = tf.nn.dropout(total_inputs, KEEP_PROB)

        with tf.variable_scope("RNN"):
#            for time_step in range(num_steps):
#                if time_step > 0: tf.get_variable_scope().reuse_variables()
            #tf.get_variable_scope().reuse_variables()  ## ????
    
            cell_output, state = tf.nn.dynamic_rnn( cell,
                                   total_inputs, self.seq_len,
                                   initial_state = states)
        
        output = tf.reshape( cell_output, [-1, self.hidden_size])  
        self.logits = tf.matmul(output, self.weight) + self.bias
        
        ## Define loss and mean cost:
        #loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
        loss = tf.nn.seq2seq.sequence_loss_by_example(
            [self.logits],
            [tf.reshape(self.targets, [-1])],
            [tf.ones([batch_size * num_steps ], dtype=tf.float32)])
        self.cost = tf.reduce_sum(loss) / batch_size
        self.final_state = state
                
        if not is_training: return
    
        trainable_variables = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, trainable_variables), MAX_GRAD_NORM)
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
        self.train_op = optimizer.apply_gradients(zip(grads, trainable_variables))
        
    def get_state_variables(self, batch_size, cell):
        # For each layer, get the initial state and make a variable out of it
        # to enable updating its value.
        state_variables = []
        for state_c, state_h in cell.zero_state(batch_size, tf.float32):
#            state_variables.append(tf.contrib.rnn.LSTMStateTuple(
            state_variables.append(tf.nn.rnn_cell.LSTMStateTuple(
                tf.Variable(state_c, trainable=False),
                tf.Variable(state_h, trainable=False)))
        # Return as a tuple, so that it can be fed to dynamic_rnn as an initial state
        return tuple(state_variables) 
    
    def get_weights(self, input_dim, output_dim, name):
        name_w = name + '_w'
        name_b = name + '_b'
        weight = tf.get_variable(name_w, [input_dim, output_dim])
        bias = tf.get_variable(name_b, [output_dim])
        return [weight, bias]

In [7]:
initializer = tf.random_uniform_initializer(-0.05, 0.05)
with tf.variable_scope("diab_model", reuse=None, initializer=initializer):
    train_model = Diab_Model(True, TRAIN_BATCH_SIZE, TRAIN_NUM_STEP,
                                 ft_len_list, TAR_SIZE, HIDDEN_SIZE)
with tf.variable_scope("diab_model", reuse=True, initializer=initializer):
    eval_model = Diab_Model(False, EVAL_BATCH_SIZE, EVAL_NUM_STEP, 
                                 ft_len_list, TAR_SIZE, HIDDEN_SIZE)
        
session = tf.Session()
#session.run( tf.global_variables_initializer()  )
session.run( tf.initialize_all_variables()  )
train_queue = one_batch(train_data, TRAIN_BATCH_SIZE,
                            TRAIN_NUM_STEP, NUM_FT, epoch_size)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=session, coord=coord)
for i in range(NUM_EPOCH):
    #t1 = time.time()
    perplexity = run_epoch(session, train_model, train_queue,
                            train_model.train_op, True, epoch_size)
    #del_t = time.time()-t1
    if i%20 ==0:
        print("In iteration: %d" % (i) )
        print("perplexity is %.3f" % (perplexity))
        #print('time spent is %.3f secs' % (del_t))
            
coord.request_stop()
coord.join(threads)

In iteration: 0
perplexity is 1.424
In iteration: 20
perplexity is 1.336
In iteration: 40
perplexity is 1.304
In iteration: 60
perplexity is 1.275
In iteration: 80
perplexity is 1.268
In iteration: 100
perplexity is 1.263
In iteration: 120
perplexity is 1.258
In iteration: 140
perplexity is 1.268
In iteration: 160
perplexity is 1.255
In iteration: 180
perplexity is 1.250
In iteration: 200
perplexity is 1.250
In iteration: 220
perplexity is 1.253
In iteration: 240
perplexity is 1.248
In iteration: 260
perplexity is 1.248
In iteration: 280
perplexity is 1.259
In iteration: 300
perplexity is 1.246
In iteration: 320
perplexity is 1.249
In iteration: 340
perplexity is 1.245
In iteration: 360
perplexity is 1.246
In iteration: 380
perplexity is 1.246
In iteration: 400
perplexity is 1.244
In iteration: 420
perplexity is 1.247
In iteration: 440
perplexity is 1.244
In iteration: 460
perplexity is 1.249
In iteration: 480
perplexity is 1.244


In [8]:
np.log(1.152)

0.14149956227369942

In [62]:
train_X.shape

(3275, 18, 149)

In [81]:
train_X[11:12, :, 47]

array([[  0.95342466,   1.58356164,   1.15068493,   1.34246575,
          1.59178082,   1.09315068,   0.95890411,   1.80273973,
          1.03561644,   0.92054795,  10.        ,  10.        ,
          1.01643836,   1.16986301,   0.8630137 ,   0.        ,
          0.        ,   0.        ]])

In [46]:
x = train_X[11:12, 8:9, :]
y = train_Y[11:12, 8:9, :]
length = train_lengths[11:12,]
print(train_X.shape, train_Y.shape, train_lengths.shape)

((3275, 18, 149), (3275, 18, 1), (3275,))


In [37]:
print(  len(num_list)   )
num_list[47]  ###  '000_forcast_yr.csv'
#cate_list

50


'000_forcast_yr.csv'

In [47]:
x[:,:,47] 

array([[[ 0.36558904,  0.05432127]]])

In [49]:
train_X[11:12, :, 47]

array([[  0.95342466,   1.58356164,   1.15068493,   1.34246575,
          1.59178082,   1.09315068,   0.95890411,   1.80273973,
          1.03561644,   0.92054795,  10.        ,  10.        ,
          1.01643836,   1.16986301,   0.8630137 ,   0.        ,
          0.        ,   0.        ]])

In [30]:
x.shape

(1, 1, 149)

In [23]:
y

array([[[ 0.]]])

In [27]:
x[0,0,47]

10.0

In [82]:
for idx in range(20):
    x[0,0,47] = 0.1 + idx* 0.5
    state, logits = session.run([eval_model.final_state, eval_model.logits],
                            feed_dict ={
                            eval_model.input_data: x,
                            eval_model.targets: y,
                            eval_model.seq_len: length })
    softmax_ = 1./(np.exp(logits[0][0] - logits[0][1])  +1.    )
    print(x[0,0,47], softmax_)

(0.10000000000000001, 0.17435222699496827)
(0.59999999999999998, 0.17435222699496827)
(1.1000000000000001, 0.17435222699496827)
(1.6000000000000001, 0.24390848686513664)
(2.1000000000000001, 0.24390848686513664)
(2.6000000000000001, 0.24390848686513664)
(3.1000000000000001, 0.24390848686513664)
(3.6000000000000001, 0.32916636615605105)
(4.0999999999999996, 0.32999470662024943)
(4.5999999999999996, 0.32999470662024943)
(5.0999999999999996, 0.32999470662024943)
(5.5999999999999996, 0.47996865439085706)
(6.0999999999999996, 0.47996865439085706)
(6.5999999999999996, 0.47996865439085706)
(7.0999999999999996, 0.47996865439085706)
(7.5999999999999996, 0.54865419576071084)
(8.0999999999999996, 0.59204466239632114)
(8.5999999999999996, 0.59204466239632114)
(9.0999999999999996, 0.59204466239632114)
(9.5999999999999996, 0.67278393441361406)


In [29]:
#state = session.run(eval_model.initial_state)
#train_data = [train_X, train_Y, train_lengths]
state, logits = session.run([eval_model.final_state, eval_model.logits],
                            feed_dict ={
                            eval_model.input_data: x,
                            eval_model.targets: y,
                            eval_model.seq_len: length })

In [30]:
logits

array([[-0.45776057,  0.48654795]], dtype=float32)

In [21]:
HIDDEN_SIZE

50

In [20]:
state[1][0].shape

(1, 50)

In [22]:
state[1][1].shape

(1, 50)

In [5]:
aaa = tf.placeholder(tf.float32, [12, 7, 60 ])
c_weights = []
for idx in range(10):
    name = 'c_weight_' + str(idx)
    c_weights.append(  get_weights(6, 11, name) )  

bbb = tf.reshape(aaa , [-1, 60])
xxx = tf.strided_slice(bbb, [0, 0], [12*7, 6], [1,1])
zzz = tf.matmul(xxx, c_weights[0][0] ) + c_weights[0][1] 

for idx in range(1, 10):
    xxx = tf.strided_slice(bbb, [0, idx*6], [12*7, (idx+1)*6], [1,1])
    yyy = tf.matmul(xxx, c_weights[idx][0] ) + c_weights[idx][1] 
    # zzz = tf.concat([zzz, yyy], axis=1) ##for py35
    zzz = tf.concat(1, [zzz, yyy])   ##for py27
zzz

<tf.Tensor 'concat_9:0' shape=(84, 110) dtype=float32>

In [16]:
for itm in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(itm.name)

diab_model/n_ft_weight:0
diab_model/n_ft_bias:0
diab_model/weight:0
diab_model/bias:0
diab_model/RNN/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Matrix:0
diab_model/RNN/RNN/MultiRNNCell/Cell0/BasicLSTMCell/Linear/Bias:0
diab_model/RNN/RNN/MultiRNNCell/Cell1/BasicLSTMCell/Linear/Matrix:0
diab_model/RNN/RNN/MultiRNNCell/Cell1/BasicLSTMCell/Linear/Bias:0


In [1]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [2]:
## create input data
len1 = 4
len2 = 1
X = np.random.randn(2,len1,5)
## set the length of the 2nd sample len2 = 1 (and pad zeros):

X[1,len2:]=0
X[1, :len2] = X[0, :len2]
seq_len = np.array([len1, len2])

In [4]:
## create a LSTM cell:
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell( num_units=3, state_is_tuple=True  )
#lstm_cell = tf.contrib.rnn.BasicLSTMCell( num_units=3, state_is_tuple=True )
seq_lengths = tf.placeholder(dtype=tf.int32, shape=[2])
## specify dtype when there's no initial states:
outputs, last_states = tf.nn.dynamic_rnn( lstm_cell, X, seq_lengths, 
                                          dtype=tf.float64)

In [6]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables() )
#sess.run(tf.global_variables_initializer() )

In [7]:
out, state = sess.run([outputs, last_states ],feed_dict={seq_lengths:
                                                        seq_len} )

In [8]:
## LSTM has two latent states  c  and  h:
print(out.shape, np.array(state).shape)

((2, 4, 3), (2, 2, 3))


In [9]:
print('lstm outputs and last h-state for seq 0:')
print( out[0])
print( state[1][0])

lstm outputs and last h-state for seq 0:
[[ 0.15113592  0.12003063 -0.00958839]
 [ 0.14691026  0.14576676 -0.32608086]
 [ 0.35864184  0.23916316 -0.23207145]
 [ 0.38370038  0.10601759 -0.24954055]]
[ 0.38370038  0.10601759 -0.24954055]


In [10]:
print('lstm outputs and last state for seq 1:')
print( out[1])
print( state[1][1])

lstm outputs and last state for seq 1:
[[ 0.15113592  0.12003063 -0.00958839]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
[ 0.15113592  0.12003063 -0.00958839]
